In [ ]:

#DataSet is available in below link
#http://stats.espncricinfo.com/ci/engine/records/team/series_results.html?id=159;type=trophy


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import time
import logging

In [2]:
def LaunchURL(URL):
    # create a new Firefox session
    driver1 = webdriver.Firefox(executable_path=r'C:\Users\shubh\Jupyter\BPL\geckodriver.exe')
    driver1.implicitly_wait(10)
    driver1.get(URL)
    return driver1

In [3]:
def SeriesPage(driver):
    return driver.find_elements_by_class_name('data-link')

In [4]:
def AllMatchDetails(driver):
    return driver.find_elements_by_class_name('potMatchLink')

In [5]:
def RedirectToCommetry(driver, Inning):
    NewURL = driver.current_url
    NewURL = NewURL.replace('/scorecard/','/commentary/')+'?innings=' +  str(Inning)
    driver.get(NewURL)
    return driver


In [6]:
def FindDivElement(driver):
    Frame = driver.find_element_by_id('pane-main')
    Frame2 = Frame.find_element_by_id('main-container')
    return Frame2.find_elements_by_class_name('main-content')[0]

In [57]:
def BatingTeamName(driver):
    return driver.find_elements_by_tag_name('button')[0].get_property('innerText')

In [59]:
def OppositionTeam(driver,Inning):
    
    if Inning ==1:
        return driver.find_elements_by_class_name('react-router-link')[13].get_property('innerText')
    else:
        return driver.find_elements_by_class_name('react-router-link')[12].get_property('innerText')

In [11]:
#launch url
logging.basicConfig(filename="newfile.log", 
                    format='%(asctime)s %(message)s', 
                    filemode='w')
logger=logging.getLogger() 
logger.setLevel(logging.DEBUG)
url = "http://stats.espncricinfo.com/ci/engine/records/team/series_results.html?id=159;type=trophy"
logger.info(url)
driverMain = LaunchURL(url)
logger.info("Main Thead Opened")

In [62]:
# get All Session data 
data = []
matchNumber =0
SeriesLinks = SeriesPage(driverMain)
logger.info("SeriesData Page Opened")
for EachSeries in SeriesLinks:
    #click on Each Series
    print ('-------------------------------------' + str(EachSeries.get_property('innerText')))
    logger.info(str(EachSeries.get_property('innerText') + " Series Started"))
    #EachSeries.click()
    SessionURL = EachSeries.get_attribute('href')
    driver = LaunchURL(SessionURL)
    logger.info ("New Thread Open")
    MatchLinks = AllMatchDetails(driver)
    for match in MatchLinks:
        # Click on Each Match
        #print (match.find_element_by_tag_name('a').get_attribute('href'))
        #match.click()
        #time.sleep(0.5)
        #matchURL = driver.current_url
        #try:
        matchNumber += 1
        driverTemp = LaunchURL(match.find_element_by_tag_name('a').get_attribute('href'))
        #driver.execute_script('window.history.go(-1)')
        # Re Direct to Commetry Tab
        Ground = driverTemp.find_elements_by_class_name('stadium-details')[0].get_property('innerText')
        for Inning in range(1,3):
        

            driverTemp = RedirectToCommetry(driverTemp, Inning)
            logger.info(str(match.find_element_by_tag_name('a').get_property('innerText'))+ " Opened in new thread")
            #Scroll Down
            #ScrollDown(driver)

            # Find Main Div Element
            div = FindDivElement(driverTemp)
            blnTaskComplete =False
            SCROLL_PAUSE_TIME = 0.5
            for i in range (0,15):
                html = driverTemp.find_element_by_tag_name('html')
                html.send_keys(Keys.END)
                time.sleep(0.1)
                html.send_keys(Keys.UP)
                time.sleep(0.1)
                html.send_keys(Keys.END)
            # get End Over Details
            # div.find_elements_by_tag_name('h4')
            # get Each Ball Details

            div = FindDivElement(driverTemp)
            for eachball in div.find_elements_by_class_name('commentary-item'):
                record = eachball.get_property('innerText')
                if ' to ' in record:
                    if str(str(record.splitlines()[0]).replace(" ","")[0]).isnumeric:
                        tempdata = str(matchNumber) + '\n' + Ground + '\n' + str(Inning) + '\n'+ BatingTeamName(div) + '\n'+ OppositionTeam(div,Inning) + '\n' + record
                        print (tempdata)
                        data.append (tempdata.splitlines())
                        logger.info ("Data Extracted" + str(record))
                    else:
                        tempdata = str(matchNumber) + '\n' + Ground + '\n' + str(Inning) + '\n'+ BatingTeamName(div) + '\n'+ OppositionTeam(div,Inning) + '\n' + record
                        print (tempdata.splitlines()[2:])
                        data.append (tempdata)

                        logger.info ("Data Extracted from 2nd element" + str(record))
        driverTemp.close()
        break
    break
        #except:
            #logger.info ("Error Occured and passed")
            #pass
logger.info ("Data Appened into DataFrame")
df = pd.DataFrame(data=data)
df.to_csv('BPL.csv')                
logger.info ("Data Saved")

-------------------------------------Bangladesh Premier League
1
Shere Bangla National Stadium, Mirpur, Dhaka
1
Royal Innings
Burnr Innings
Match State: Innings Break

19.6	
6
Yasir Arafat to Styris, SIX runs,
1
Shere Bangla National Stadium, Mirpur, Dhaka
1
Royal Innings
Burnr Innings
19.5	
1
Yasir Arafat to Sohail Tanvir, 1 run,
1
Shere Bangla National Stadium, Mirpur, Dhaka
1
Royal Innings
Burnr Innings
19.4	
W
Yasir Arafat to Alok Kapali, OUT,

Alok Kapali b Yasir Arafat 56 (46m 36b 6x4 2x6) SR: 155.55
1
Shere Bangla National Stadium, Mirpur, Dhaka
1
Royal Innings
Burnr Innings
19.3	
1
Yasir Arafat to Styris, 1 run,
1
Shere Bangla National Stadium, Mirpur, Dhaka
1
Royal Innings
Burnr Innings
19.2	
W
Yasir Arafat to Trego, OUT,

PD Trego b Yasir Arafat 62 (66m 54b 7x4 1x6) SR: 114.81
1
Shere Bangla National Stadium, Mirpur, Dhaka
1
Royal Innings
Burnr Innings
19.1	
0
Yasir Arafat to Trego, no run,
1
Shere Bangla National Stadium, Mirpur, Dhaka
1
Royal Innings
Burnr Innings
18.6	
1
G

1
Shere Bangla National Stadium, Mirpur, Dhaka
1
Royal Innings
Burnr Innings
4.1	
0
Gayle to Imrul Kayes, no run,
1
Shere Bangla National Stadium, Mirpur, Dhaka
1
Royal Innings
Burnr Innings
3.6	
1
Harwood to Imrul Kayes, 1 run,
1
Shere Bangla National Stadium, Mirpur, Dhaka
1
Royal Innings
Burnr Innings
3.5	
1
Harwood to Trego, 1 run,
1
Shere Bangla National Stadium, Mirpur, Dhaka
1
Royal Innings
Burnr Innings
3.4	
1
Harwood to Imrul Kayes, 1 run,
1
Shere Bangla National Stadium, Mirpur, Dhaka
1
Royal Innings
Burnr Innings
3.3	
1
Harwood to Trego, 1 run,
1
Shere Bangla National Stadium, Mirpur, Dhaka
1
Royal Innings
Burnr Innings
3.2	
0
Harwood to Trego, no run,
1
Shere Bangla National Stadium, Mirpur, Dhaka
1
Royal Innings
Burnr Innings
3.1	
1lb
Harwood to Imrul Kayes, 1 leg bye,
1
Shere Bangla National Stadium, Mirpur, Dhaka
1
Royal Innings
Burnr Innings
2.6	
0
Yasir Arafat to Trego, no run,
1
Shere Bangla National Stadium, Mirpur, Dhaka
1
Royal Innings
Burnr Innings
2.5	
0
Yasir Ar

1
Shere Bangla National Stadium, Mirpur, Dhaka
2
Burnr Innings
Royal Innings
11.6	
1w
Hodge to Alok Kapali, 1 wide,
1
Shere Bangla National Stadium, Mirpur, Dhaka
2
Burnr Innings
Royal Innings
11.5	
1
Hodge to Trego, 1 run,
1
Shere Bangla National Stadium, Mirpur, Dhaka
2
Burnr Innings
Royal Innings
11.4	
1
Hodge to Alok Kapali, 1 run,
1
Shere Bangla National Stadium, Mirpur, Dhaka
2
Burnr Innings
Royal Innings
11.3	
1
Hodge to Trego, 1 run,
1
Shere Bangla National Stadium, Mirpur, Dhaka
2
Burnr Innings
Royal Innings
11.2	
1
Hodge to Alok Kapali, 1 run,


In [41]:
div.find_elements_by_tag_name('button')[0].get_property('innerText')

'Royal Innings'

In [56]:
div.find_elements_by_class_name('react-router-link')[13].get_property('innerText')

'Burnr Innings'